In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from jwlab.cluster_analysis import prep_cluster_analysis
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from scipy import stats
from sklearn.svm import LinearSVC


In [2]:
length_per_window = 10
num_sliding_windows = int(1200/ length_per_window)

In [3]:
participants = ["105"]

In [ ]:
participants = ["904", "905", "906", "908", "909", "912","913", "914", "916", "917", "919",\
                "920", "921", "923", "924", "927", "928", "929", "930", "932"]

In [ ]:
participants = ["105", "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]


In [ ]:
participants = ["105", "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124",\
               "904", "905", "906", "908", "909", "912","913", "914", "916", "917", "919",\
                "920", "921", "923", "924", "927", "928", "929", "930", "932"]


In [4]:
Xwin, ywin, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging", length_per_window=length_per_window)
#Xwin, ywin, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials", length_per_window=length_per_window)
#Xwin, ywin, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants", length_per_window=length_per_window)


loaded


In [ ]:
ywin[0]
np.amax(ywin[0])
unique, counts = np.unique(ywin[0], return_counts=True)
dict(zip(unique, counts))

In [7]:
len(Xwin)

120

In [ ]:
ywin[0].shape

In [8]:
num_sliding_windows

120

In [ ]:
# # Cross validation

#X, y, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging", length_per_window=length_per_window)

# for j in range(num_sliding_windows):
#     #model = SVC(gamma=.001, kernel = 'rbf', C=1000)
#     model = LinearSVC(C=1e-9, max_iter=5000)
#     cv_results = cross_validate(model, X[j], y[j], cv=num_folds)
#     if j in results.keys(): 
#         results[j] += cv_results['test_score'].tolist()
#     else:
#         results[j] = cv_results['test_score'].tolist()
    

# for i in range(num_sliding_windows):
#     assert len(results[i]) == num_iterations * num_folds

In [9]:
# Cross validation with RepeatedKFold

num_iterations = 3
num_folds = 5


import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score



results = {}
for j in range(num_sliding_windows):
    X = Xwin[j]
    y = ywin[j]

    #model = SVC(gamma=.001, kernel = 'rbf', C=1e-4)
    model = LinearSVC(C=1e-5, max_iter=5000)
    rkf = RepeatedKFold(n_splits=num_folds, n_repeats=num_iterations, random_state=2652124)
    for train_index, test_index in rkf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        testScore = accuracy_score(y_test,y_pred)

        if j in results.keys(): 
            results[j].append(testScore)
        else:
            results[j]=[]
            results[j].append(testScore)
    

for i in range(num_sliding_windows):
    assert len(results[i]) == num_iterations * num_folds

ValueError: Found input variables with inconsistent numbers of samples: [35, 28]

In [ ]:
X.shape

In [ ]:
X_train.shape

In [ ]:
scoreMean = []
stdev = []

for i in range(num_sliding_windows):
    scoreMean.append(np.mean(results[i]))
    stdev.append(np.std(results[i]))



In [ ]:
np.mean(scoreMean)

In [ ]:
scoreMean

In [ ]:
#stdev

In [ ]:
# T-test
accuracy_by_guessing = [0.5] * (num_iterations * num_folds)
pvalues = []
for i in range(num_sliding_windows):
    istat = stats.ttest_1samp(results[i], .5)
    pvalues += [istat.pvalue] if istat.statistic > 0 else [1]

In [ ]:
# Finding contiguous time cluster
valid_window = [i for i,v in enumerate(pvalues) if v <= 0.025]
print(valid_window)